이 노트북은 [케라스 창시자에게 배우는 딥러닝 2판](https://tensorflow.blog/kerasdl2/)의 예제 코드를 담고 있습니다.

<table align="left">
    <tr>
        <td>
            <a href="https://colab.research.google.com/github/rickiepark/deep-learning-with-python-2nd/blob/main/chapter11_part04_sequence-to-sequence-learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
        </td>
    </tr>
</table>

## 텍스트 분류를 넘어: 시퀀스-투-시퀀스 학습

### 기계 번역 예제

In [1]:
!wget http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip
!unzip -q spa-eng.zip

--2024-05-28 01:30:17--  http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.202.207, 173.194.203.207, 74.125.199.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.202.207|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2638744 (2.5M) [application/zip]
Saving to: ‘spa-eng.zip’

spa-eng.zip         100%[===================>]   2.52M  --.-KB/s    in 0.01s   

2024-05-28 01:30:17 (244 MB/s) - ‘spa-eng.zip’ saved [2638744/2638744]



In [2]:
text_file = "spa-eng/spa.txt"
with open(text_file) as f:
    lines = f.read().split("\n")[:-1]
text_pairs = []
for line in lines:
    english, spanish = line.split("\t")
    spanish = "[start] " + spanish + " [end]"
    text_pairs.append((english, spanish))

In [3]:
import random
print(random.choice(text_pairs))

('The boy entered by the back door.', '[start] El chico entró por la puerta trasera. [end]')


In [4]:
import random
random.shuffle(text_pairs)
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples:num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples:]

**영어와 스페인어 텍스트 쌍을 벡터화하기**

In [5]:
import tensorflow as tf
from tensorflow.keras import layers
import string
import re

strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(
        lowercase, f"[{re.escape(strip_chars)}]", "")

vocab_size = 15000
sequence_length = 20

source_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
)
target_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
    standardize=custom_standardization,
)
train_english_texts = [pair[0] for pair in train_pairs]
train_spanish_texts = [pair[1] for pair in train_pairs]
source_vectorization.adapt(train_english_texts)
target_vectorization.adapt(train_spanish_texts)

**번역 작업을 위한 데이터셋 준비하기**

In [6]:
batch_size = 64

def format_dataset(eng, spa):
    eng = source_vectorization(eng)
    spa = target_vectorization(spa)
    return ({
        "english": eng,
        "spanish": spa[:, :-1],
    }, spa[:, 1:])

def make_dataset(pairs):
    eng_texts, spa_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    spa_texts = list(spa_texts)
    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, spa_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset, num_parallel_calls=4)
    return dataset.shuffle(2048).prefetch(16).cache()

train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

In [7]:
for inputs, targets in train_ds.take(1):
    print(f"inputs['english'].shape: {inputs['english'].shape}")
    print(f"inputs['spanish'].shape: {inputs['spanish'].shape}")
    print(f"targets.shape: {targets.shape}")

inputs['english'].shape: (64, 20)
inputs['spanish'].shape: (64, 20)
targets.shape: (64, 20)


In [8]:
for inputs, targets in train_ds.take(1):
    print(f"inputs['english'].shape: {inputs['english'][0]}")
    print(f"inputs['spanish'].shape: {inputs['spanish'][0]}")
    print(f"targets.shape: {targets[0]}")

inputs['english'].shape: [   6 1253   24   28  119    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0]
inputs['spanish'].shape: [   2    8   15 1892   17    5  129   91    3    0    0    0    0    0
    0    0    0    0    0    0]
targets.shape: [   8   15 1892   17    5  129   91    3    0    0    0    0    0    0
    0    0    0    0    0    0]


### RNN을 사용한 시퀀스-투-시퀀스 모델

**기반 인코더**

In [9]:
from tensorflow import keras
from tensorflow.keras import layers

embed_dim = 256
latent_dim = 1024

source = keras.Input(shape=(None,), dtype="int64", name="english")
x = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(source)
encoded_source = layers.Bidirectional(
    layers.GRU(latent_dim), merge_mode="sum")(x)

**GRU 기반 디코더와 엔드-투-엔드 모델**

In [10]:
past_target = keras.Input(shape=(None,), dtype="int64", name="spanish")
x = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(past_target)
decoder_gru = layers.GRU(latent_dim, return_sequences=True)
x = decoder_gru(x, initial_state=encoded_source)
x = layers.Dropout(0.5)(x)
target_next_step = layers.Dense(vocab_size, activation="softmax")(x)
seq2seq_rnn = keras.Model([source, past_target], target_next_step)

**RNN 기반 시퀀스-투-시퀀스 모델 훈련하기**

In [11]:
seq2seq_rnn.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])
seq2seq_rnn.fit(train_ds, epochs=15, validation_data=val_ds)

Epoch 1/15
1302/1302 [==============================] - 133s 90ms/step - loss: 4.6851 - accuracy: 0.3184 - val_loss: 3.9167 - val_accuracy: 0.3853
Epoch 2/15
1302/1302 [==============================] - 107s 82ms/step - loss: 3.7290 - accuracy: 0.4152 - val_loss: 3.2712 - val_accuracy: 0.4673
Epoch 3/15
1302/1302 [==============================] - 107s 82ms/step - loss: 3.2120 - accuracy: 0.4730 - val_loss: 2.8841 - val_accuracy: 0.5151
Epoch 4/15
1302/1302 [==============================] - 108s 83ms/step - loss: 2.8577 - accuracy: 0.5128 - val_loss: 2.6355 - val_accuracy: 0.5500
Epoch 5/15
1302/1302 [==============================] - 108s 83ms/step - loss: 2.5844 - accuracy: 0.5459 - val_loss: 2.4591 - val_accuracy: 0.5764
Epoch 6/15
1302/1302 [==============================] - 108s 83ms/step - loss: 2.3617 - accuracy: 0.5742 - val_loss: 2.3277 - val_accuracy: 0.5966
Epoch 7/15
1302/1302 [==============================] - 108s 83ms/step - loss: 2.1770 - accuracy: 0.5988 - val_loss: 2

KeyboardInterrupt: 

**RNN 인코더와 디코더로 새로운 문장 번역하기**

In [ ]:
import numpy as np
spa_vocab = target_vectorization.get_vocabulary()
spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))
max_decoded_sentence_length = 20

def decode_sequence(input_sentence):
    tokenized_input_sentence = source_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = target_vectorization([decoded_sentence])
        next_token_predictions = seq2seq_rnn.predict(
            [tokenized_input_sentence, tokenized_target_sentence])
        sampled_token_index = np.argmax(next_token_predictions[0, i, :])
        sampled_token = spa_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token
        if sampled_token == "[end]":
            break
    return decoded_sentence

test_eng_texts = [pair[0] for pair in test_pairs]
for _ in range(20):
    input_sentence = random.choice(test_eng_texts)
    print("-")
    print(input_sentence)
    print(decode_sequence(input_sentence))

-
Nice timing.
1/1 [==============================] - 0s 21ms/step
[start] se [UNK] [end]
-
The old man sat on the bench with his eyes closed.
1/1 [==============================] - 0s 21ms/step
[start] el anciano se sentó en la cama con los ojos cerrados [end]
-
She must be angry.
1/1 [==============================] - 0s 20ms/step
[start] ella debe estar enojada [end]
-
I'll be a few minutes late.
1/1 [==============================] - 0s 21ms/step
[start] estaré tarde con unos minutos [end]
-
Which one is mine?
1/1 [==============================] - 0s 21ms/step
[start] cuál es mío [end]
-
What you said was very interesting.
1/1 [==============================] - 0s 22ms/step
[start] lo que dijiste realmente era muy interesante [end]
-
I go to bed very early.
1/1 [==============================] - 0s 21ms/step
[start] me voy a la cama temprano [end]
-
Tom sensed that Mary was in pain.
1/1 [==============================] - 0s 20ms/step
[start] tom se [UNK] a mary que estaba en pelig

### 트랜스포머를 사용한 시퀀스-투-시퀀스 모델

#### 트랜스포머 디코더

**`TransformerDecoder` 클래스**

In [ ]:
class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

    def call(self, inputs, encoder_outputs, mask=None):
        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            use_causal_mask=True)
        attention_output_1 = self.layernorm_1(inputs + attention_output_1)
        attention_output_2 = self.attention_2(
            query=attention_output_1,
            value=encoder_outputs,
            key=encoder_outputs
        )
        attention_output_2 = self.layernorm_2(
            attention_output_1 + attention_output_2)
        proj_output = self.dense_proj(attention_output_2)
        return self.layernorm_3(attention_output_2 + proj_output)

#### 기계 번역을 위한 트랜스포머

**`PositionalEmbedding` 층**

In [ ]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

    def get_config(self):
        config = super(PositionalEmbedding, self).get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config

**엔드-투-엔드 트랜스포머**

In [ ]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]
        attention_output = self.attention(
            inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

In [ ]:
embed_dim = 256
dense_dim = 2048
num_heads = 8

encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="english")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)

decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="spanish")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, dense_dim, num_heads)(x, encoder_outputs)
x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)
transformer = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

**시퀀스-투-시퀀스 트랜스포머 훈련하기**

In [ ]:
transformer.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])
transformer.fit(train_ds, epochs=30, validation_data=val_ds)

Epoch 1/30
1302/1302 [==============================] - 57s 39ms/step - loss: 3.9515 - accuracy: 0.4188 - val_loss: 2.7509 - val_accuracy: 0.5770
Epoch 2/30
1302/1302 [==============================] - 43s 33ms/step - loss: 2.5898 - accuracy: 0.6019 - val_loss: 2.2324 - val_accuracy: 0.6471
Epoch 3/30
1302/1302 [==============================] - 43s 33ms/step - loss: 2.2178 - accuracy: 0.6515 - val_loss: 2.0779 - val_accuracy: 0.6707
Epoch 4/30
1302/1302 [==============================] - 43s 33ms/step - loss: 2.0307 - accuracy: 0.6788 - val_loss: 2.0129 - val_accuracy: 0.6806
Epoch 5/30
1302/1302 [==============================] - 43s 33ms/step - loss: 1.9215 - accuracy: 0.6963 - val_loss: 1.9987 - val_accuracy: 0.6851
Epoch 6/30
1302/1302 [==============================] - 42s 33ms/step - loss: 1.8491 - accuracy: 0.7085 - val_loss: 1.9938 - val_accuracy: 0.6901
Epoch 7/30
1302/1302 [==============================] - 43s 33ms/step - loss: 1.8019 - accuracy: 0.7176 - val_loss: 2.0158 -

**트랜스포머 모델을 사용해 새로운 문장 번역하기**

In [ ]:
import numpy as np
spa_vocab = target_vectorization.get_vocabulary()
spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))
max_decoded_sentence_length = 20

def decode_sequence(input_sentence):
    tokenized_input_sentence = source_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = target_vectorization(
            [decoded_sentence])[:, :-1]
        predictions = transformer(
            [tokenized_input_sentence, tokenized_target_sentence])
        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = spa_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token
        if sampled_token == "[end]":
            break
    return decoded_sentence

test_eng_texts = [pair[0] for pair in test_pairs]
for _ in range(20):
    input_sentence = random.choice(test_eng_texts)
    print("-")
    print(input_sentence)
    print(decode_sequence(input_sentence))

-
The pain was almost unbearable.
[start] el dolor era [UNK] casi rica [end]
-
I have to go to the toilet.
[start] tengo que ir al baño [end]
-
He looked around the waiting room.
[start] Él miró a su alrededor de la sala [end]
-
Tom pulled Mary out of the burning car.
[start] tom sacó a mary el coche se está en francia [end]
-
They had wonderful lives.
[start] ellos lo hicieron vive [end]
-
Keep in mind that smoking is not good for your health.
[start] sigue usted de fumar es una buena para la salud [end]
-
I'm three years older than my brother.
[start] yo soy tres años mayor que mi hermano [end]
-
Come home with me.
[start] ven a mi casa conmigo [end]
-
Strange things happened on her birthday.
[start] tan raro pasó cosas en su cumpleaños [end]
-
Two against one is not a fair fight.
[start] dos es uno de no es solo una lloviendo [end]
-
Will you have some more pie?
[start] te [UNK] algo más de torta [end]
-
Put some clothes on.
[start] [UNK] algo de ropa [end]
-
I think I still have ti

## 요약